<center><h2>Traitement Automatique du LAngage au service du GRANd Débat National</h2></center>

Fork du projet TALAGRAND disponible ici : https://github.com/Quantmetry/grand-debat  
Mots clés : **NLP, TAL, LDA, Embeddings, TextRank**

In [1]:
import warnings; warnings.simplefilter('ignore')

# Téléchargement des données
-  Récupérer la liste des thèmes en analysant le site du Grand Débat
-  Sélectionner un thème en modifiant la variable `selected_theme`
-  Télécharger ensuite le fichier json le plus récent associé à un thème depuis datagouv.fr

In [2]:
from grand_debat.downloading import download_data, get_themes

In [3]:
themes = get_themes() # not working
list(themes.keys())

[]

In [4]:
themes = {"1 L'organisation de l'État et des services publics": 
              "http://opendata.auth-6f31f706db6f4a24b55f42a6a79c5086.storage.sbg.cloud.ovh.net/"+
              "2019-03-21/ORGANISATION_DE_LETAT_ET_DES_SERVICES_PUBLICS.json",
          "2 la-transition-ecologique": 
              "http://opendata.auth-6f31f706db6f4a24b55f42a6a79c5086.storage.sbg.cloud.ovh.net/2019-03-21/"+
              "LA_TRANSITION_ECOLOGIQUE.json"
         }

In [5]:
# Choisir le numéro associé au thème que l'on souhaite étudier
selected_theme = 2

In [6]:
#download_data(themes, selected_theme)

# Chargement des données
-  Charger le fichier json précédemment téléchargé
-  Afficher les différentes questions associées au thème sélectionné
-  Sélectionner une question à étudier (variable `selected_question`)
-  Afficher le nombre de réponses existantes correspondant à cette question

In [7]:
from grand_debat.loading import load_data, load_question, get_path

In [8]:
path = get_path(themes, selected_theme)
data_theme_json = load_data(path)

In [9]:
questions_id = [r['questionId'] for r in data_theme_json[0]['responses']]
questions_title = [r['questionTitle'] for r in data_theme_json[0]['responses']]
for id, title in zip(questions_id, questions_title):
    print("ID : {} - Question: {}".format(id, title))
    print()

ID : UXVlc3Rpb246MTYw - Question: Quel est aujourd'hui pour vous le problème concret le plus important dans le domaine de l'environnement ?

ID : UXVlc3Rpb246MTYx - Question: Que faudrait-il faire selon vous pour apporter des réponses à ce problème ?

ID : UXVlc3Rpb246MTQ2 - Question: Diriez-vous que votre vie quotidienne est aujourd'hui touchée par le changement climatique ?

ID : UXVlc3Rpb246MTQ3 - Question: Si oui, de quelle manière votre vie quotidienne est-elle touchée par le changement climatique ?

ID : UXVlc3Rpb246MTQ4 - Question: À titre personnel, pensez-vous pouvoir contribuer à protéger l'environnement ?

ID : UXVlc3Rpb246MTQ5 - Question: Si oui, que faites-vous aujourd'hui pour protéger l'environnement et/ou que pourriez-vous faire ?

ID : UXVlc3Rpb246MTUw - Question: Qu'est-ce qui pourrait vous inciter à changer vos comportements comme par exemple mieux entretenir et régler votre chauffage, modifier votre manière de conduire ou renoncer à prendre votre véhicule pour de tr

In [10]:
# Indiquer ici l'identifiant associé à la question étudiée
selected_question = 'UXVlc3Rpb246MTQ3' # Si oui, de quelle manière votre vie quotidienne est-elle touchée ...

In [11]:
selected_question = str(selected_question)
data_theme_response_dict = load_question(questions_id, data_theme_json, selected_question)
print("Nombre de réponses analysées :", len(data_theme_response_dict[selected_question]))

Nombre de réponses analysées : 92792


In [27]:
import pandas as pd
def inspect_responses(data_theme_json, selected_question):
    resp_flat = pd.DataFrame.from_records(
        pd.json_normalize(data_theme_json[:10])["responses"].explode().values)
    return resp_flat.loc[resp_flat["questionId"] == selected_question, "value"].dropna()

list(inspect_responses(data_theme_json, selected_question))

["Pollution de l'air, pollution de nos aliments, maladie chronique dans mon entourage, maladie digestive",
 "Réchauffement climatique évident , températures en hausse permanente et changements brutaux d'un point de vue météo avec trés gros orages . La question est mal posée à mon avis .",
 'Climat',
 "la qualité de l'air n'est pas la même selon la météo"]

## <h2>Détection de thèmes</h2>
-  Détecter les thèmes principaux associés à la question sélectionnée en utilisant l'algorithme **LDA** (Latent Dirichlet Allocation)
-  Le resultat de l'analyse se trouve dans le fichier pyLDAVIS_tf.html

In [13]:
# spacy preparation
import spacy
nlp = spacy.load("fr_core_news_md")

In [61]:
from tqdm import tqdm
import importlib
import grand_debat.theme_detection as td
importlib.reload(td)

<module 'grand_debat.theme_detection' from '/Users/charlesprat/RepoGit/grand-debat/grand_debat/theme_detection.py'>

In [62]:
from numpy.random import default_rng
rng = default_rng(seed=32)

In [75]:
answs = rng.choice(data_theme_response_dict[selected_question], size=10000, replace=False).tolist()

In [76]:
answ_lems = td.lemmatize_answers(answs, nlp)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:14<00:00, 677.15it/s]


In [77]:
tf_vectorizer, lda_tf = td.get_themes(answ_lems, n_topics=5, stop_words=nlp.Defaults.stop_words)

print("Les différents thèmes associés à la question se situent dans le fichier pyLDAVIS_tf.html")

iteration: 1 of max_iter: 100
iteration: 2 of max_iter: 100
iteration: 3 of max_iter: 100
iteration: 4 of max_iter: 100, perplexity: 608.6675
iteration: 5 of max_iter: 100
iteration: 6 of max_iter: 100
iteration: 7 of max_iter: 100
iteration: 8 of max_iter: 100, perplexity: 565.0849
iteration: 9 of max_iter: 100
iteration: 10 of max_iter: 100
iteration: 11 of max_iter: 100
iteration: 12 of max_iter: 100, perplexity: 551.4498
iteration: 13 of max_iter: 100
iteration: 14 of max_iter: 100
iteration: 15 of max_iter: 100
iteration: 16 of max_iter: 100, perplexity: 543.9126
iteration: 17 of max_iter: 100
iteration: 18 of max_iter: 100
iteration: 19 of max_iter: 100
iteration: 20 of max_iter: 100, perplexity: 539.0742
iteration: 21 of max_iter: 100
iteration: 22 of max_iter: 100
iteration: 23 of max_iter: 100
iteration: 24 of max_iter: 100, perplexity: 536.1698
iteration: 25 of max_iter: 100
iteration: 26 of max_iter: 100
iteration: 27 of max_iter: 100
iteration: 28 of max_iter: 100, perplexi

In [78]:
lambda_rel = 0
for ii in range(lda_tf.components_.shape[0]):
    print(td.get_topic_by_relevance(tf_vectorizer, lda_tf, lambd=lambda_rel, n_topic=ii))

['hiver' 'chaud' 'froid' 'hausse' 'prix' 'travail' 'climatisation' 'doux'
 'chauffage']
['canicule' 'inondation' 'tempête' 'fréquent' 'violent' 'pluie'
 'phénomène' 'vent' 'épisode']
['pollution' 'air' 'allergie' 'qualité' 'problème' 'respiratoire'
 'produit' 'asthme' 'déchet']
['disparition' 'insecte' 'oiseau' 'espèce' 'biodiversité' 'animal'
 'abeille' 'diminution' 'perte']
['saison' 'changement' 'dérèglement' 'climat' 'catastrophe' 'modification'
 'monde' 'faune' 'février']


/Users/charlesprat/miniconda3/envs/tf/lib/python3.7/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/charlesprat/miniconda3/envs/tf/lib/python3.7/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/charlesprat/miniconda3/envs/tf/lib/python3.7/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/charlesprat/miniconda3/envs/tf/lib/python3.7/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/charlesprat/miniconda3/en

## <h2>Résumé de chaque thème</h2>
-  Résumer un grand thème en utilisant TextRank, technique de résumé extractive
-  Spécifier le nombre de phrases que l'on souhaite pour former le résumé dans la variable `sn`
-  Les résultats se situent dans le fichier demo.docx

In [ ]:
from grand_debat.text_summarization import prepare_data, apply_page_rank_algorithm, create_titles
from grand_debat.vectorization import sentence_to_vec
from grand_debat.clusterisation import apply_clusterisation

In [ ]:
#import nltk
#nltk.download('punkt')

Transformer le document texte en un ensemble de phrases dont la typographie est standardisée

In [ ]:
clean_sentences = prepare_data(data_theme_response_dict, selected_question)

Récupérer la phrase la plus caractéristique de chaque thème

In [ ]:
sentences_mixture_topics, title_sentences, titles = create_titles(clean_sentences, tf_vectorizer, lda_tf)

Associer à chaque phrase du corpus son thème (issu de **LDA**) le plus carctéristique

In [ ]:
sentences_paragraph = apply_clusterisation(sentences_mixture_topics)

Représenter chaque phrase sous la forme d'un vecteur dense. Ces vecteurs sont obtenus à partir d'embeddings entraînés à partir de l'ensemble des contributions au Grand Débat

In [ ]:
sentence_vectors, word_embeddings = sentence_to_vec(clean_sentences, file_model='data/word2vec.10k.100d.txt')

On applique alors l'algorithme **TextRank** à l'ensemble des phrases d'un thème pour en extraire les phrases carctéristiques

In [ ]:
# Indiquer le nombre de phrases qui formeront le résumé
sn = 10
apply_page_rank_algorithm(clean_sentences, sentences_paragraph, word_embeddings, sn)

## Export de la synthèse sous la forme d'un document word
-  Créer le document final contenant les résumés pour chaque thème de la question.
-  Le processus prend environ 20 minutes sur une machine équipée d'un intel core i5.

In [ ]:
from grand_debat.saving import create_final_doc, get_theme, get_question

In [ ]:
theme_sentence = get_theme(themes, selected_theme)
question_sentence = get_question(questions_id, questions_title, selected_question)
create_final_doc(title_sentences, titles, clean_sentences,
                 sentences_paragraph, word_embeddings, theme_sentence, question_sentence)